# Fine tuning the model to make a chat bot

This is the big guacamole at the end of the rainbow. We'll be fine tuning one of the OpenAI models to be able to respond sort of like ChatGPT. I think there's an example of trying to do this on the foundation model in `openai.ipynb` without fine-tuning, and right now it _sucks_.

In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

import import_ipynb
import openai # type:ignore
import gpt # type:ignore
import torch
import urllib
import ssl
import os
import json
from pprint import pprint
from typing import TypedDict
from torch.utils.data import Dataset, DataLoader
import tiktoken
from functools import partial
import textwrap
from datasets import load_dataset

def get_device() -> torch.device:
    if torch.cuda.is_available(): # type: ignore[attr-defined]
        return torch.device("cuda")
    elif torch.backends.mps.is_available(): # type: ignore[attr-defined]
        return torch.device("mps:0")
    else:
        return torch.device("cpu")

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
env: CUDA_LAUNCH_BLOCKING=1
env: CUBLAS_WORKSPACE_CONFIG=:4096:8


2025-06-26 23:09:13.323715: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-26 23:09:13.331811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751004553.341949    8599 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751004553.345094    8599 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751004553.352792    8599 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Download the instruction training data

This is 1,100 instruction-response pairs (actually some have a third field called input) that were made specifically for the book.

In [2]:
class InstructionExample(TypedDict):
    instruction: str  # A description of the task to be performed
    input: str        # Optional parameter for the task
    output: str       # The expected result of performing the task

def download_and_load_file(file_path: str, url: str) -> list[InstructionExample]:
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url, context=ssl_context) as response: # type:ignore
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
            text_data = file.read()

    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    return data

file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))
print("Example:")
pprint(data[1])

Number of entries: 1100
Example:
{'input': 'He go to the park every day.',
 'instruction': 'Edit the following sentence for grammar.',
 'output': 'He goes to the park every day.'}


## Convert the examples to Stanford Alpaca format

The [format](https://github.com/tatsu-lab/stanford_alpaca) looks like this:

```
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
```

Or, if there's no input:

```
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
```

In [3]:
def format_input(entry: InstructionExample, include_response:bool=True) -> str:
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry["input"]}" if entry["input"] else ""
    response_text = f"\n\n### Response:\n{entry["output"]}" if include_response else ""

    return instruction_text + input_text + response_text

train_portion = int(len(data) * 0.85)
test_portion = int(len(data) * 0.1)
val_portion = len(data) - train_portion - test_portion

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

In [4]:
class InstructionDataset(Dataset):
    def __init__(self, data: list[InstructionExample], tokenizer: tiktoken.Encoding):
        self.data = data

        # Pre-tokenize texts
        self.encoded_texts = []
        for entry in data:
            full_text = format_input(entry)
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )
    
    def __getitem__(self, index) -> list[int]:
        return self.encoded_texts[index]
    
    def __len__(self):
        return len(self.data)


## Custom collate function

Passing in a custom collate function lets us easily pad out shorter sequences in each batch to match the longest one.
Initially, the padding token will be `<|endoftext|>`, but we'll eventually set it up so that there's only one EOT token
and the padding will be done with `-100`.

The collate function is responsible for:
1. Finding the longest sequence in the batch
2. Padding and preparing inputs
3. Removing the extra EOT tokens
4. Converting the token list to a tensor and transferring it to the target device.


### We're not masking the instructions

We could use `-100` to mask out the instructions from each example. That would avoid rewarding the model for memorizing
worthless bits like "Below is a task…", and some people think that's helpful. But it's controversial, and there's at least
one paper, ["Instruction Tuning with Loss Over Instructions,"](https://arxiv.org/abs/2405.14394) that argues that it's
better to train on the whole thing.

Maybe I'll try adding instruction masking later, but for now it's not recommended.

In [5]:
def custom_collate_fn(
        batch: list[list[int]],
        pad_token_id: int=50256, # i.e., <|endoftext|>
        ignore_index: int=-100, # this is the default ignore index for torch.nn.CrossEntropyLoss
        allowed_max_length: int|None=None,
        device: str|torch.device="cpu"
) -> tuple[torch.Tensor, torch.Tensor]:
    batch_max_length = max([len(item)+1 for item in batch])

    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        new_item += [pad_token_id]
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])
        targets = torch.tensor(padded[1:])

        mask = targets == pad_token_id # tensor([bool * max_length])
        indices = torch.nonzero(mask).squeeze() # type:ignore
        if indices.numel() > 1:
            # Note: we only do this -100 thing in the targets tensor
            targets[indices[1:]] = ignore_index

        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]
        
        inputs_lst.append(inputs)
        targets_lst.append(targets)

    inputs_tensor = torch.stack(inputs_lst)
    targets_tensor = torch.stack(targets_lst)

    return inputs_tensor.to(device), targets_tensor.to(device)

customized_collate_fn = partial(custom_collate_fn, device=get_device(), allowed_max_length=1024)

## Create the Datasets and DataLoaders

As a reminder:
- **Dataset**: a class that exposes `__getitem__` and `__len__`, so it's like
  a list or a vector. It's not really specialized for anything in particular, it's
  just a convenient way to wrap data from some source.
- **DataLoader**: a class that encapsulates logic for ordering (shuffle or not),
  associating inputs with targets, batching, parallelization, etc.

In [6]:
num_workers = 0
batch_size = 8

gpt.manual_seed(123)
tokenizer = tiktoken.get_encoding("gpt2")

train_dataset = InstructionDataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn, # ah, makes sense how partial would be useful now
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

val_dataset = InstructionDataset(val_data, tokenizer)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

test_dataset = InstructionDataset(test_data, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

In [7]:
def train_model_on_small_example_set(model: gpt.SimplifiedGPT):
    example_prompt = format_input({
        'instruction': "Convert this sentence to passive voice.",
        'input': 'The chef cooked the meal.',
        'output': ''}, include_response=True)

    training_config: gpt.GPTTrainingConfig = gpt.DEFAULT_TRAINING_CONFIG.copy()
    training_config['gradient_clipping'] = False
    training_config['epochs'] = 2
    training_config['peak_lr'] = 5e-5
    training_config['initial_lr'] = 4e-6
    training_config['eval_frequency'] = 20

    config = openai.GPT_CONFIG_355M.copy()

    trainer = gpt.GPTModel(cfg=config, training_cfg=training_config, model=model)

    trainer.train_loader(train_loader, val_loader, example_prompt, run_name="learning rate 5e-5, 2 epochs")
    return trainer

# gpt355m = openai.load_openai_model(openai.GPT_CONFIG_355M, "355M").model
# trainer = train_model_on_small_example_set(gpt355m)

In [8]:
def instruct(trainer: gpt.GPTModel, instruction: str, input='', temperature=0.8):
    prompt = format_input({
        'instruction': instruction,
        'input': input,
        'output': '',
    }, include_response=True)
    result = trainer.prompt(prompt, max_tokens=1024, temperature=temperature)
    result = result[len(prompt):]
    print(textwrap.fill(result, width=120))

In [9]:
# Should output: The healthier of the two foods is carrots.

# instruct(trainer, "Determine which of the two foods is healthier", "carrots, lard")

# Training on Alpaca

The [tatsu-lab/alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca) dataset is about 52k rows, so almost 50x bigger than the one we just trained on. That should give 50x better results, right???

In [10]:
alpaca = load_dataset("tatsu-lab/alpaca", split='train')

In [11]:
def token_len(txt: str):
    tks = tokenizer.encode(txt)
    return len(tks)

alpaca: list[InstructionExample] = [x for x in alpaca if token_len(x['text']) <= 323] # type: ignore

In [12]:
batch_size = 1

alpaca_train_portion = int(len(alpaca) * 0.95)
alpaca_val_portion = int(len(alpaca) * 0.002) # about 100 examples
alpaca_test_portion = len(alpaca) - alpaca_train_portion - alpaca_val_portion

alpaca_train_data = alpaca[:alpaca_train_portion]
alpaca_test_data = alpaca[alpaca_train_portion:alpaca_train_portion + alpaca_test_portion]
alpaca_val_data = alpaca[alpaca_train_portion + alpaca_test_portion:]

alpaca_train_dataset = InstructionDataset(alpaca_train_data, tokenizer)
alpaca_test_dataset = InstructionDataset(alpaca_test_data, tokenizer)
alpaca_val_dataset = InstructionDataset(alpaca_val_data, tokenizer)

print(f"Train: {len(alpaca_train_data)}")
print(f"Val: {len(alpaca_val_data)}")
print(f"Test: {len(alpaca_test_data)}")

alpaca_train = DataLoader(
    alpaca_train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

alpaca_test = DataLoader(
    alpaca_test_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

alpaca_val = DataLoader(
    alpaca_val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

Train: 48915
Val: 102
Test: 2473


In [13]:
def train_model_on_big_example_set(model: gpt.SimplifiedGPT):
    example_prompt = format_input({
        'instruction': "Convert this sentence to passive voice.",
        'input': 'The chef cooked the meal.',
        'output': ''}, include_response=True)

    training_config: gpt.GPTTrainingConfig = gpt.DEFAULT_TRAINING_CONFIG.copy()
    training_config['gradient_clipping'] = False
    training_config['epochs'] = 2
    training_config['peak_lr'] = 5e-5
    training_config['initial_lr'] = 4e-6
    training_config['eval_frequency'] = 200

    config = openai.GPT_CONFIG_355M.copy()

    trainer = gpt.GPTModel(cfg=config, training_cfg=training_config, model=model)

    trainer.train_loader(alpaca_train, alpaca_val, example_prompt, run_name="alpaca less validation, smaller samples, cache clear")
    return trainer

gpt355m = openai.load_openai_model(openai.GPT_CONFIG_355M, "355M").model
trainer = train_model_on_big_example_set(gpt355m)

File already exists and is up-to-date: gpt2/355M/checkpoint
File already exists and is up-to-date: gpt2/355M/encoder.json
File already exists and is up-to-date: gpt2/355M/hparams.json
File already exists and is up-to-date: gpt2/355M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/355M/model.ckpt.index
File already exists and is up-to-date: gpt2/355M/model.ckpt.meta
File already exists and is up-to-date: gpt2/355M/vocab.bpe
355M model loaded.
{'training_loss': 3.6558, 'validation_loss': 3.2972, 'tokens_seen': 108, 'example_output': '\nThe chef was happy.\n\n### Out', 'learning_rate': 4e-06}
{'training_loss': 0.922, 'validation_loss': 1.5381, 'tokens_seen': 20409, 'example_output': 'This sentence will be passive. It will not engage', 'learning_rate': 4.188081365634263e-06}
{'training_loss': 1.4134, 'validation_loss': 1.4989, 'tokens_seen': 40900, 'example_output': 'The chef cooked the meal', 'learning_rate': 4.3761627312685265e-06}
{'training_loss': 1.2112, 'va

In [17]:
trainer.save(name="fine-tuned-355m-alpaca")

In [25]:
llm = trainer

In [16]:
import numpy as np

lengths = np.array(example_lengths)
print(f"Count: {len(lengths)}")
print(f"Min: {lengths.min()}, Max: {lengths.max()}")
print(f"Mean: {lengths.mean():.2f}, Std: {lengths.std():.2f}")
print(f"Median: {np.median(lengths)}")
print(f"90th percentile: {np.percentile(lengths, 90)}")
print(f"95th percentile: {np.percentile(lengths, 95)}")
print(f"99th percentile: {np.percentile(lengths, 99)}")

Count: 51490
Min: 36, Max: 323
Mean: 108.14, Std: 49.45
Median: 95.0
90th percentile: 174.0
95th percentile: 203.0
99th percentile: 276.0


In [57]:
instruct(llm, "Answer the following question honestly.", "Are you conscious?")

Yes, I am conscious.
